<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bert-score-evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets bert-score tqdm

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from bert_score import score
import torch

t5_small_paradetox_1Token_tokenizer = AutoTokenizer.from_pretrained("HamdanXI/t5-small-paradetox-1Token-split-masked")
t5_small_paradetox_1Token_model = AutoModelForSeq2SeqLM.from_pretrained("HamdanXI/t5-small-paradetox-1Token-split-masked")

# BART-base-detox (10,000 epochs with the learning rate of 3e-5)
bart_base_detox_tokenizer = AutoTokenizer.from_pretrained("s-nlp/bart-base-detox")
bart_base_detox_model = AutoModelForSeq2SeqLM.from_pretrained("s-nlp/bart-base-detox")

bart_base_paradetox_split_tokenizer = AutoTokenizer.from_pretrained("HamdanXI/bart-base-paradetox-split")
bart_base_paradetox_split_model = AutoModelForSeq2SeqLM.from_pretrained("HamdanXI/bart-base-paradetox-split")


paradetox_dataset = load_dataset("HamdanXI/paradetox-split")
paradetox_1token_dataset = load_dataset("HamdanXI/paradetox-1Token-Split")

In [3]:
def max_token_length(input, label, tokenizer):
  max_token_length_input = max(len(tokenizer.encode(item)) for item in input)
  max_token_length_label = max(len(tokenizer.encode(item)) for item in label)

  if max_token_length_input > max_token_length_label:
      highest_length = max_token_length_input
  else:
      highest_length = max_token_length_label

  return highest_length

In [4]:
from tqdm import tqdm

def generate_predictions(texts, model, tokenizer, highest_length):
    predictions = []
    for text in tqdm(texts, desc="Generating predictions"):
        inputs = tokenizer(text, padding=True, truncation=True, max_length=highest_length, return_tensors="pt")
        with torch.no_grad():
            outputs = model.generate(**inputs)
        predictions.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])
    return predictions

def bert_score_evaluate(input, label, model, tokenizer, highest_length):
  predictions = generate_predictions(input, model, tokenizer, highest_length)

  # Compute BERT Score
  P, R, F1 = score(predictions, label, lang="en", rescale_with_baseline=True)

  # Compute and print the average scores
  print(f"Precision: {P.mean()}, Recall: {R.mean()}, F1 Score: {F1.mean()}")

In [5]:
highest_length_1token_t5_small = max_token_length(paradetox_1token_dataset['test']["en_toxic_comment"], paradetox_1token_dataset['test']["en_neutral_comment"], t5_small_paradetox_1Token_tokenizer)
highest_length_1token_bart_base = max_token_length(paradetox_1token_dataset['test']["en_toxic_comment"], paradetox_1token_dataset['test']["en_neutral_comment"], bart_base_detox_tokenizer)
highest_length_1token_bart_base_split = max_token_length(paradetox_1token_dataset['test']["en_toxic_comment"], paradetox_1token_dataset['test']["en_neutral_comment"], bart_base_paradetox_split_tokenizer)

In [ ]:
bert_score_evaluate(paradetox_1token_dataset['test']["en_toxic_comment"], paradetox_1token_dataset['test']["en_neutral_comment"], t5_small_paradetox_1Token_model, t5_small_paradetox_1Token_tokenizer, highest_length_1token_t5_small)

Generating predictions:   0%|          | 0/811 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating predictions: 100%|██████████| 811/811 [02:49<00:00,  4.79it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.89088374376297, Recall: 0.8717513680458069, F1 Score: 0.8811372518539429


In [ ]:
bert_score_evaluate(paradetox_1token_dataset['test']["en_toxic_comment"], paradetox_1token_dataset['test']["en_neutral_comment"], bart_base_detox_model, bart_base_detox_tokenizer, highest_length_1token_bart_base)

Generating predictions:   0%|          | 0/811 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating predictions: 100%|██████████| 811/811 [06:23<00:00,  2.11it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8873310685157776, Recall: 0.8524273633956909, F1 Score: 0.8696255683898926


In [7]:
bert_score_evaluate(paradetox_1token_dataset['test']["en_toxic_comment"], paradetox_1token_dataset['test']["en_neutral_comment"], bart_base_paradetox_split_model, bart_base_paradetox_split_tokenizer, highest_length_1token_bart_base_split)

Generating predictions:   0%|          | 0/811 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating predictions: 100%|██████████| 811/811 [05:56<00:00,  2.28it/s]


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8960155248641968, Recall: 0.8626090884208679, F1 Score: 0.8791258335113525


In [8]:
highest_length_t5_small = max_token_length(paradetox_dataset['test']["en_toxic_comment"], paradetox_dataset['test']["en_neutral_comment"], t5_small_paradetox_1Token_tokenizer)
highest_length_bart_base = max_token_length(paradetox_dataset['test']["en_toxic_comment"], paradetox_dataset['test']["en_neutral_comment"], bart_base_detox_tokenizer)
highest_length_bart_base_split = max_token_length(paradetox_dataset['test']["en_toxic_comment"], paradetox_dataset['test']["en_neutral_comment"], bart_base_paradetox_split_tokenizer)

In [9]:
bert_score_evaluate(paradetox_dataset['test']["en_toxic_comment"], paradetox_dataset['test']["en_neutral_comment"], t5_small_paradetox_1Token_model, t5_small_paradetox_1Token_tokenizer, highest_length_t5_small)

Generating predictions:   0%|          | 0/671 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating predictions: 100%|██████████| 671/671 [02:36<00:00,  4.30it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.5166414976119995, Recall: 0.5892131328582764, F1 Score: 0.5521959066390991


In [ ]:
bert_score_evaluate(paradetox_dataset['test']["en_toxic_comment"], paradetox_dataset['test']["en_neutral_comment"], bart_base_detox_model, bart_base_detox_tokenizer, highest_length_bart_base)

In [11]:
bert_score_evaluate(paradetox_dataset['test']["en_toxic_comment"], paradetox_dataset['test']["en_neutral_comment"], bart_base_paradetox_split_model, bart_base_paradetox_split_tokenizer, highest_length_bart_base_split)

Generating predictions: 100%|██████████| 671/671 [05:16<00:00,  2.12it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.598376452922821, Recall: 0.6236093044281006, F1 Score: 0.6103842854499817
